<a href="https://colab.research.google.com/github/BrithneyHernandez25/BrithneyHernandez.github.io/blob/main/CODIGO_NOTICIAS_MEXICO_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GoogleNews
from GoogleNews import GoogleNews
import pandas as pd
import datetime
import re
import gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default
from datetime import datetime
creds, _ = default()
gc = gspread.authorize(creds)

terminos_busqueda = [
                     'Betty, La Fea: La Historia Continúa', 'Betty, La Fea','Betty La Fea',"Yo soy Betty, la fea",
                     "Mi Lady Jane",
                     "Elle Woods", "Legalmente Rubia",
                     "Rojo, Blanco y Sangre Azul","Red, White & Royal Blue 2",
                     'The Boys',
                     "Road House",
                     "Culpa Tuya",
                     "I am Celine Dion",
                     'Sueltos en Los Cabos',
                     "Cada Minuto Cuenta",
                     "Courtois: La vuelta del número 1","Thibaut Courtois",
                    "Los Anillos de Poder","The Rings of Power",
                     "temptation island",
                     "Federer",
                     "Yakuza",

                     ]
googlenews = GoogleNews(lang='es', region='MX')
todos_resultados = []

# Convertir términos de búsqueda a minúsculas para normalización
terminos_busqueda = [termino.lower() for termino in terminos_busqueda]

for termino in terminos_busqueda:
    googlenews.search(termino)
    resultados = []
    for pagina in range(1, 4):
        googlenews.getpage(pagina)
        resultados.extend(googlenews.result())
    todos_resultados.extend(resultados)

df = pd.DataFrame(todos_resultados)

# Convertir descripciones y títulos a minúsculas para comparación
df['desc'] = df['desc'].str.lower()
df['title'] = df['title'].str.lower()

# Filtrar resultados que contengan alguno de los términos de búsqueda
resultados_finales = df[df['desc'].str.contains('|'.join(terminos_busqueda))]

def remover_parte_variable(url):
    return re.sub(r'&ved=.+', '', url)

resultados_finales['link'] = resultados_finales['link'].apply(remover_parte_variable)
resultados_unicos = resultados_finales.drop_duplicates(subset=['link'])
def categorizar_titulo(titulo, descripcion):
    if any(keyword in titulo for keyword in ["Paco", "Stanley", "Beli", "Paola Durante", "Paola Duarte", "¿Quién lo mató?","Diego Boneta","bezares"]):
        return "¿Quién lo mató?"
    elif any(keyword in titulo for keyword in ["I Am: Céline Dion", "Céline Dion", "Celine Dion", "Céline Dion", "I AM: CELINE DION"]):
        return "I Am: Céline Dion"
    elif any(keyword in titulo for keyword in ["Betty", "la fea"]):
        return "Betty, La Fea: La Historia Continúa"
    elif any(keyword in titulo for keyword in ["anillos", "The Lord of the Rings", "Anillos"]):
        return "The Lord of the Rings S2"
    elif any(keyword in titulo for keyword in ["Culpa Tuya", "Culpa tuya"]):
        return "Culpa Tuya"
    elif any(keyword in titulo for keyword in ["The Boys", "Karl Urban", "Antony Star"]):
        return "The Boys S4"
    elif any(keyword in titulo for keyword in ["Fallout", "Ella Purnell", "Walton Goggins"]):
        return "Fallout  S1"
    elif any(keyword in titulo for keyword in ["idea of you", "The Idea of You", "La Idea de Ti", "Anne Hathaway", "The idea of you",'The Idea of You',"'The Idea of You'"]):
        return "The Idea of You"
    elif any(keyword in titulo for keyword in ["Maxton", "Maxton Hall"]):
        return "Maxton Hall"
    elif any(keyword in titulo for keyword in ["Sueltos en Los Cabos", "sueltos en Los Cabos", "Niurka", "El Capi", "Kristal Silva", "Sueltos en los Cabos", "Sueltos en los cabos"]):
        return "Sueltos en Los Cabos S1"
    elif any(keyword in titulo for keyword in ["Legalmente Rubia", "Reese Witherspoon", "Elle","Legalmente Rubia"]):
        return "Legalmente Rubia"
    elif any(keyword in titulo for keyword in ["Mi lady Jane", "Mi lady Jane", "Mi lady Jane"]):
        return "Mi lady Jane"
    elif any(keyword in titulo for keyword in ["Road House", "Road House", "Road House"]):
        return "Mi lady Jane"
    elif any(keyword in titulo for keyword in ["Red, White & Royal Blue 2", "Rojo, Blanco y Sangre Azul"]):
        return "Red, White & Royal Blue 2"
    else:
        return None


# Filtrar las filas donde media es de otro pais
medias_a_quitar = ["Vandal", "20Minutos","Hello Friki","QueVer","El Periódico","Neeo", "Mundo Deportivo","Revista OHLALÁ!","La Cuarta",  "Gamereactor", "OkDiario",
                   "Cinemascomics.com","Lovingseries","Diez Minutos","YouTube","VADER","El Diario NY","La Casa De EL","Ciudad Magazine","Viví el Oeste",
                   "La República","vemosTV","Dailymotion","Revista Velvet","CanalRGZ","El Mundo","No es cine todo lo que reluce","Tiempo Latino","La Opinión",
                   "Radio Canal 95","La Casa de EL","Diario AS","El Comercio Perú","CulturaOcio","Moyens.net","La Prensa Gráfica"]

# Añadir filtros de dominios no deseados
def categorizar_dominio(url):
    dominios_excluir = ['.es', '.cl', '.arg',".co",".cr",".pr",".pe",".ni"]
    return any(url.endswith(dominio) for dominio in dominios_excluir)

resultados_unicos = resultados_unicos[~resultados_unicos['media'].isin(medias_a_quitar)]
resultados_unicos = resultados_unicos[~resultados_unicos['link'].apply(categorizar_dominio)]

def categorizar_pais(texto):
    if any(pais in texto.lower() for pais in ["usa", "peru","perú" "colombia", "ecuador", "euro", "españa","español","nicaragua","ciudad real","ny"]):
        return "Quitar"
    else:
        return ""

# Mantener la columna media y filtrar las filas sin eliminarla
resultados_unicos['pais_categoria'] = resultados_unicos['media'].apply(lambda x: categorizar_pais(x))
resultados_unicos = resultados_unicos[resultados_unicos['pais_categoria'] != "Quitar"]

# Aplicar la categorización del título
resultados_unicos['categoria'] = resultados_unicos.apply(lambda x: categorizar_titulo(x['title'], x['desc']), axis=1)

# Quitar "Hace" de la columna 'date'
def limpiar_fecha(fecha):
    return fecha.replace('Hace ', '')

resultados_unicos['date'] = resultados_unicos['date'].apply(limpiar_fecha)

# Mantener solo las columnas necesarias
columnas_mantener = ['title', 'media', 'desc', 'link', 'categoria', 'date']
resultados_unicos = resultados_unicos[columnas_mantener]
resultados_unicos = resultados_unicos.astype(str)

# Abriendo la hoja de cálculo
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1QS_4jeK0HuX67YYD3aVcYpgxttSQVWjJh5uYT1g6JuM/edit#gid=1459345472")
worksheet_noticias = spreadsheet.worksheet("Noticias")

# Pega los nuevos datos en la fila 2, columna A
worksheet_noticias.update('A2', resultados_unicos.values.tolist(), value_input_option='RAW')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 8.5 MB/s eta 0:00:00


<ipython-input-1-ab95d1438081>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados_finales['link'] = resultados_finales['link'].apply(remover_parte_variable)
<ipython-input-1-ab95d1438081>:136: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_noticias.update('A2', resultados_unicos.values.tolist(), value_input_option='RAW')


{'spreadsheetId': '1QS_4jeK0HuX67YYD3aVcYpgxttSQVWjJh5uYT1g6JuM',
 'updatedRange': 'Noticias!A2:F220',
 'updatedRows': 219,
 'updatedColumns': 6,
 'updatedCells': 1314}